In [ ]:
!nvidia-smi

Sat Mar 30 16:53:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
model_id = "google/gemma-2b-it"
# model_id = "google/gemma-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request and answers the question.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [ ]:
result = get_completion(query="explain what kubernetes is in detail", model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request and answers the question.
  explain what kubernetes is in detail
  
model


    Sure, here's a detailed explanation of what Kubernetes is:

**What is Kubernetes?**

Kubernetes (Kubernetes) is an open-source platform for managing containerized applications. It automates the deployment, scaling, and removal of containerized applications across multiple physical or virtual machines. Kubernetes is not a single piece of software but rather a collection of tools and resources that work together to provide a unified solution for managing containerized applications.

**Key Features of Kubernetes:**

* **Cluster Management:** Kubernetes manages a cluster of one or more nodes, each running a containerized application.
* **Pod Management:** Kubernetes creates and manages pods, which are isolated units of resource allocation within the cluster.
* **Service Discovery:** Kubernetes auto

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ahmedgongi/Devops_LLM", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
df = dataset.to_pandas()
df.head(10)

,Response,Prompt,Instruction
0,Fixed it with these simpler steps ;)Reset Shar...,Visual Studio 2017 HRESULT: 0x80004004,I was perfectly running an ASP.NET Core projec...
1,get an account on docker hub.https://hub.docke...,how to package a docker image in a single file,"I have a 5GB docker image named ""ubuntu-dev-up..."
2,"Great question. I did some research and, as fa...",How do I unset a Docker image label?,I've a Dockerfile starting with theofficial ng...
3,Is this what you looking for?docker image insp...,List Docker images for ARM from the CLI,I own a Mac M1 and I run Docker on it.\nOn OSX...
4,I found this old issue today using docker comp...,When running a Django dev server with docker/f...,I'm writing a Dockerfile which needs to run mu...
5,"apt-getgenerally needs to run as root, but onc...","Why does simple Dockerfile give ""permission de...","I am learning to use Docker with ROS, and I am..."
6,Using your Dockerfile with my project I added ...,"Poetry install throws ""Connection pool is full...","I cannot build my docker image. It throws ""Con..."
7,"You need to advertise yourKafkabroker askafka,...",Spring Boot containers can not connect to the ...,I'm trying to use microservicesSpring Bootwith...
8,But one service can not interact with another ...,Can (or should) 2 docker containers interact w...,"We're dockerizing our micro services app, and ..."
9,"Volumes are treated as mounts in Docker, which...",expose files from docker container to host,I have a docker container that holds a django ...


In [ ]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a question. Write a response that ' \
               'appropriately completes the request and answers the question honestly and politely.\n\n'
    # Samples with additional context into.
    if data_point['Prompt']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["Instruction"]} here is the question {data_point["Prompt"]} <end_of_turn>\n<start_of_turn>model{data_point["Response"]} <end_of_turn>"""
    # Without
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["Instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["Response"]} <end_of_turn>"""
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("Questions", text_column)

In [ ]:
dataset = dataset.shuffle(seed=9114)  # Shuffle
dataset = dataset.map(lambda samples: tokenizer(samples["Questions"]), batched=True)

Map:   0%|          | 0/142731 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
train_df = train_data.to_pandas()
train_df.head(10)

,Response,Prompt,Instruction,Questions,input_ids,attention_mask
0,I wonder how come Docker did not complain abou...,"package in requirements.txt, but not seen in d...","I have valid requirements.txt file, but docker...",<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,One common solution to achieve auditing is to ...,Access console contents,Is if possible for a running bash session to a...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Try:wget -O /dev/null -q http://mydomain.com/m...,wget to call a php script but it creates output,I am callingwgetfromcrontabto periodically cal...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,\nproperty Item someItem: Item { ... }\n\nThe ...,QML object property memory management,"\nIf I have some object, Item or QtObject as a...",<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"I'm worried, however, that merging\n changes ...",Moving from SVN to HG : branching and backup,My company runs svn right now and we are very ...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,You need to manually trigger a new deployment ...,How to add grafana's admin password in values ...,I want to ask how to add Grafana admin passwor...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,\n\n\n\n\n\n\n\n 5\n \n\n\n\...,"How to merge two branches and, automatically, ...",\nI have a MEAN stack project separated into t...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,Deploying Laravel app in cPanel is quite simpl...,Laravel Deployment on Shared Hosting - 404 Error,I am trying to deploy my laravel 5.1 applicati...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,"Realized that I needed to remove ""/etc/apt/apt...",How to preserve apt-cache archive directory wh...,The use case is fairly straightforward. I am ...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,"\nTricky to do in a portable way, it would lik...",Java - how to determine the current load,\nHow would I determine the current server loa...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

['gate_proj', 'k_proj', 'up_proj', 'q_proj', 'o_proj', 'v_proj', 'down_proj']


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


In [ ]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="Questions",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=5,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/114184 [00:00<?, ? examples/s]

Map:   0%|          | 0/28547 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
model.config.use_cache = False  # silence the warnings.
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,4.739200
2,4.559100
3,4.224100
4,3.820700
5,4.178400


TrainOutput(global_step=5, training_loss=4.304314422607422, metrics={'train_runtime': 57.9271, 'train_samples_per_second': 0.345, 'train_steps_per_second': 0.086, 'total_flos': 87646481080320.0, 'train_loss': 4.304314422607422, 'epoch': 0.0})

In [ ]:
new_model = "gemma-2bI-Terraforge-Finetune"

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer.padding_side = "left"

In [ ]:
result = get_completion(query="What does AssumeRole in IAM means?", model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request and answers the question.
  What does AssumeRole in IAM means?
  
model


  AssumeRole means to temporarily assume the permissions and identity of another IAM user or role. An IAM Role is a user or object that has been granted specific permissions, and the AssumeRole function allows you to temporarily grant those permissions to another IAM user.  

AssumeRole can be used to grant or revoke permissions before they are applied to the target user. This allows you to test out role changes and ensure that the new role has the desired effects before applying it to the target user or object.


In [ ]:

merged_model.push_to_hub("jainam02/gemma-2bI-Terraforge-Finetune", use_temp_dir=False)
tokenizer.push_to_hub("jainam02/gemma-2bI-Terraforge-Finetune", use_temp_dir=False)

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jainam02/gemma-2bI-Terraforge-Finetune/commit/dd59a7e5ba9b3f6496a20a71096459c87e336f1f', commit_message='Upload tokenizer', commit_description='', oid='dd59a7e5ba9b3f6496a20a71096459c87e336f1f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip3 install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.4 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl si

SyntaxError: positional argument follows keyword argument (<ipython-input-39-c2f31e213e65>, line 3)